This function computes closed null geodesics of $ C_{t_0}^{t_N}(\mathbf{x})-\lambda^2 \mathbf{I} $, where $ C_{t_0}^{t_N}(\mathbf{x}) = \begin{pmatrix} C^{11} && C^{12} \\ C^{12} && C^{22} \end{pmatrix} $ is the (symmetric) Cauchy Green strain tensor.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| mu | float | $ \mu $|
| interp_phi_prime | interpolant | $ \dot{\phi} $|
| d_threshold | float | threshold distance between start of closed null-geodesic and maximum allowed first return distance to starting point |
| interp_DOE | interpolant | $ \sin(2\phi)[C^{22}(\mathbf{x})-C^{11}(\mathbf{x})]+2\cos(2\phi)C^{12}(\mathbf{x}) $ |
| solODE_closed_curve | list | list containing the closed solution curves ($ \mathbf{x}, \phi $) |
| x0lam | list | list containing x-coordinates of initial conditions $ x_0(\lambda) $ |
| y0lam | list | list containing y-coordinates of initial conditions $ x_0(\lambda) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# Import math symbols
from math import sqrt, pi

# function computing initial conditions (depending on \mu)
from ipynb.fs.defs.init_level_set import _init_level_set

# geodesics differential equation
from ipynb.fs.defs.geodesic_equation import _geodesic_equation

# find closed curve
from ipynb.fs.defs.closed_curve import _closed_curve

# RK4 integrator
from ipynb.fs.defs.RK4_integration import RK4_integration

# tqdm shows progress bar
from tqdm.notebook import tqdm

# library for Polygon, Point objects
from shapely.geometry import Polygon, Point

# Import (cubic) RectBivariateSpline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [3]:
def _closed_null_geodesics(X, Y, lam, interp_phi_prime, d_threshold, C11, interp_DOE):
    
    # domain where the rate of strain field is defined
    defined_domain = np.isfinite(C11).astype(int)
    
    # compute initial conditions
    x0lam, y0lam, phi0lam = _init_level_set(X, Y, C11, lam)
    
    # define integration domain of dummy variable
    s = [0, 10]
    
    # define resolution of trajectories
    s_eval = np.linspace(s[0], s[1], 150)
    
    ds = s_eval[1]-s_eval[0]

    # store solutions in this list
    solODE_closed_curves = []
    
    # number of initial conditions
    len_x0lam = len(x0lam)
    
    # line needed to print tqdm
    print(' ', end='', flush=True)
    
    # iterate over all initial conditions [x0mu, y0mu, phi0mu]
    for j in tqdm(range(len_x0lam)):
        
        # initial condition
        y0 = np.array([x0lam[j], y0lam[j], phi0lam[j]])
        
        sol = np.zeros((len(s_eval), 3))
        
        # Integrate trajectory using RK4 with fixed step size
        for s in range(len(s_eval)):
            sol[s, :] = y0
            y0 = RK4_integration(s, y0, ds, interp_phi_prime, X, Y, defined_domain, interp_DOE)
        
        # store x, y, phi
        x = sol[:,0]
        y = sol[:,1]
        phi = sol[:,2]
        
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance.
        x_closed_geodesic, y_closed_geodesic, phi_closed_geodesic = _closed_curve(x, y, phi, np.array(y0), d_threshold)
        
        # store solutions
        solODE_closed_curves.append([x_closed_geodesic, y_closed_geodesic, phi_closed_geodesic])
    
    return solODE_closed_curves, [x0lam, y0lam]